In [1]:
import pandas as pd
import pypfopt as pfop

In [ ]:
# data importation
# sequence of data are corresponding correctly with the name sequence, whichever horizontally or vertically
AssetName = ["us_bond", "us_equity", "developed_equity", "emerging_equity", "hedge_fund", "reits", "commodity", "gold", "btc"]
r = [0.0204, 0.119, 0.0517, 0.0286, 0.0414, 0.0695, -0.0039, 0.0307, 7.2988]
std = [0.0598, 0.1643, 0.1503, 0.1686, 0.0641, 0.1634, 0.2277, 0.1388, 15.6325]

# covariance table
cov = [[0.0033,0.0040,0.0028,0.0044,0.0012,0.0043,-0.0049,0.0041,-0.0737],
[0.0040, 0.0248, 0.0203, 0.0183, 0.0090, 0.0153, 0.0042, 0.0004, 0.9619],
[0.0028, 0.0203, 0.0207, 0.0195, 0.0079, 0.0115, 0.0069, 0.0023, 0.8156],
[0.0044, 0.0183, 0.0195, 0.0261, 0.0081, 0.0116, 0.0041, 0.0114, -0.0785],
[0.0012, 0.0090, 0.0079, 0.0081, 0.0038, 0.0047, 0.0028, 0.0018, 0.2155],
[0.0043, 0.0153, 0.0115, 0.0116, 0.0047, 0.0245, 0.0068, 0.0007, -0.2240],
[-0.0049, 0.0042, 0.0069, 0.0041, 0.0028, 0.0068, 0.0475, 0.0016, 0.0124],
[0.0041, 0.0004, 0.0023, 0.0114, 0.0018, 0.0007, 0.0016, 0.0177	-1.1228],
[-0.0737, 0.9619, 0.8156, -0.0785, 0.2155, -0.2240, 0.0124, -1.1228, 224.0111]]

# correlation table
corr = [[1,0.44,0.34,0.48,0.35,0.48,-0.39,0.54,-0.09],
[0.44,	1,	0.90,	0.72,	0.93,	0.62,	0.12,	0.02,	0.41],
[0.34,	0.90,	1,	0.84,	0.89,	0.51,	0.22,	0.12,	0.38],
[0.48,	0.72,	0.84,	1,	0.82,	0.46,	0.12,	0.53,	-0.03],
[0.35,	0.93,	0.89,	0.82,	1,	0.49,	0.21,	0.22,	0.23],
[0.48,	0.62,	0.51,	0.46,	0.49,	1,	0.20,	0.03,	-0.10],
[-0.39,	0.12,	0.22,	0.12,	0.21,	0.20,	1,	0.05,	0.00],
[0.54,	0.02,	0.12,	0.53,	0.22,	0.03,	0.05,	1,	-0.56],
[-0.09,	0.41,	0.38,	-0.03,	0.23,	-0.10,	0.00,	-0.56,	1]]



In [ ]:
# data pre-processing